In [0]:
%python
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "d0471621-cb16-4841-ab1b-e9c0fc3370ef",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="azure-kv-scope",key="sp-secret"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/25235581-c428-4ef3-a446-dddcdfada476/oauth2/token"}
if not dbutils.fs.ls("/mnt/"):
  dbutils.fs.mount(
    source = "abfss://bhoojal@bhoojalstorageaccount.dfs.core.windows.net/",
    mount_point = "/mnt",
    extra_configs = configs)   

if not dbutils.fs.ls("/mnt/web"):
  dbutils.fs.mount(
  source = "abfss://$web@bhoojalstorageaccount.dfs.core.windows.net/",
  mount_point = "/mnt/web",
  extra_configs = configs)  
  
#Ref: https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/adls-gen2/azure-datalake-gen2-sp-access

In [0]:
%python
import json
import pandas as pd
import math

fileName = dbutils.widgets.get('FileName')
gprData = spark.read.option("multiline", "true").json("/mnt/raw/" + fileName)

gprData = gprData.toPandas()
finalData = []

for index, row in gprData.iterrows(): 
  quarter = row['scannedIn']
  city = row['city']
  for observation in row['observations']:
    
    depth = (float(observation['waveTravelTime']) * 0.3)/(math.sqrt(float(row['dielectric'])))
    print(depth)
    depth = round(depth)
    for i in range(depth):
      finalData.append(observation)
      
finaldf = pd.DataFrame(finalData, columns=['lat', 'lng', 'waveTravelTime'])

json_str = finaldf.to_json(orient="records")
blobDF = pd.DataFrame([json_str],columns=['result'])

sparkDF=spark.createDataFrame(blobDF)
#sparkDF=spark.createDataFrame(finaldf) 
print(sparkDF.count())

sparkDF.coalesce(1).write.mode("overwrite").format('json').save("dbfs:/mnt/web/temp") #write the transformed file to temp dir as ADB creates processed file with weird name

x = [f for f in dbutils.fs.ls("dbfs:/mnt/web/temp") if "part-00000" in f.name] 
dbutils.fs.mv(x[0].path, "dbfs:/mnt/web/heatmaps/" + city + "/" + quarter + ".json") #move the file from temp to another folder so that we can rename the transformed file
dbutils.fs.rm("dbfs:/mnt/web/temp", True) #remove temp folder as it contains other files created by ADB